In [1]:
from pathlib import Path
import random

import numpy as np
import torch

import data_utils
import model
import training

Задача - улучшение качества записи речи. На вход подаётся запись речи, содержащая шумы, а на выходе ожидается запись без шумов.

Для решения этой задачи применялись генеративно-состязательные сети, состоящие из генератора, вычисляющего решение и дискриминатора, оценивающего его. При этом в задаче очищения речи существуют эталонные решения и метрики, позволяющие оценить сходство решения с эталоном, и оценка решения генератором не соответствует метрикам. Идея MetricGAN - вместо классификации решений на образцовые и сгенерированные обучить генератор приближать целевую метрику.

В данном эксперименте уменьшенный вариант модели из статьи обучается на задаче восстановления записи речи. Используется датасет LibriSpeech и четыре вида шума из датасета DEMAND. Как и в статье, шум добавляется с отношением сигнал-шум от -8дб до 8дб с шагом 4 дб. Для оценки используется метрика STOI.

In [2]:
DATA_PATH = Path('data')
ls_speaker_dirs = list((DATA_PATH / 'train-clean-100/LibriSpeech/train-clean-100/').iterdir())

In [3]:
ls_train_size = int(len(ls_speaker_dirs) * .6)
ls_val_size = int(len(ls_speaker_dirs) * .2)
ls_train_dirs = ls_speaker_dirs[:ls_train_size]
ls_val_dirs = ls_speaker_dirs[ls_train_size:ls_train_size + ls_val_size]
ls_test_dirs = ls_speaker_dirs[ls_train_size + ls_val_size:]

In [4]:
DEMAND_DIR = DATA_PATH / 'DEMAND'
DEMAND_TYPES = 'NPARK', 'OOFFICE', 'PSTATION', 'SPSQUARE'
DEMAND_TYPE_DIRS = [DEMAND_DIR / type_name for type_name in DEMAND_TYPES]

In [5]:
noiser = data_utils.DemandNoiser(DEMAND_TYPE_DIRS, np.linspace(-8, 8, 5))

train_ds = data_utils.LibreSpeechDataset(ls_train_dirs, noiser)
val_ds = data_utils.LibreSpeechDataset(ls_val_dirs, noiser, random_noise=False)
test_ds = data_utils.LibreSpeechDataset(ls_test_dirs, noiser, random_noise=False)

Поскольку при обучении на всех данных эпоха занимает слишком много времени, используются случайные подмножества тренировочной и валидационной выборок. Тренировочное подмножество каждый раз меняется, чтобы избежать переобучения.

In [6]:
class RandomSubsetDataset(torch.utils.data.Dataset):
    def __init__(self, source_dataset, n, fix):
        super().__init__()
        self.source_dataset = source_dataset
        self.subset = random.sample(list(range(len(source_dataset))), n) if fix else None
        self.fix = fix
        self.n = n
        
    def __getitem__(self, index):
        if index >= self.n:
            raise IndexError
        if self.fix:
            return self.source_dataset[index]
        return self.source_dataset[random.randint(0, len(self.source_dataset) - 1)]
        
    def __len__(self):
        return self.n

In [7]:
device = 'cuda'

In [8]:
gen = model.MetricGenerator().to(device)
disc = model.MetricDiscriminator().to(device)

gen_opt = torch.optim.Adam(gen.parameters())
disc_opt = torch.optim.Adam(disc.parameters())

In [9]:
train_sample = RandomSubsetDataset(train_ds, 1250, False)
val_sample = RandomSubsetDataset(val_ds, 250, True)
training.train_gan(gen, disc, gen_opt, disc_opt, train_sample, val_sample, 5, device)

  0%|                                                                                         | 0/1250 [00:00<?, ?it/s]

Epoch 1 tr_disc_loss: 0.0271 val_disc_loss: 0.0117 tr_gen_loss: 0.0093 val_gen_loss: 0.0066, gen_stoi: 0.2247


  0%|                                                                                         | 0/1250 [00:00<?, ?it/s]

Epoch 2 tr_disc_loss: 0.0246 val_disc_loss: 0.0053 tr_gen_loss: 0.3442 val_gen_loss: 0.3383, gen_stoi: 0.6372


  0%|                                                                                         | 0/1250 [00:00<?, ?it/s]

Epoch 3 tr_disc_loss: 0.0108 val_disc_loss: 0.0043 tr_gen_loss: 0.0792 val_gen_loss: 0.0730, gen_stoi: 0.6381


  0%|                                                                                         | 0/1250 [00:00<?, ?it/s]

Epoch 4 tr_disc_loss: 0.0134 val_disc_loss: 0.0075 tr_gen_loss: 0.0137 val_gen_loss: 0.0041, gen_stoi: 0.6288


100%|████████████████████████████████████████████████████████████████████████████████| 250/250 [01:43<00:00,  2.42it/s]

Epoch 5 tr_disc_loss: 0.0435 val_disc_loss: 0.0265 tr_gen_loss: 0.0675 val_gen_loss: 0.0582, gen_stoi: 0.6366


In [10]:
torch.save(gen.state_dict(), 'gen.pt')
torch.save(disc.state_dict(), 'disc.pt')

In [12]:
test_loss, test_stoi = training.val_generator(gen, disc, test_ds, device, 1)

100%|██████████████████████████████████████████████████████████████████████████████| 5742/5742 [37:11<00:00,  2.57it/s]


In [14]:
print(f'Test loss: {test_loss:.4f} test STOI: {test_stoi:.4f}')

Test loss: 0.0605 test STOI: 0.6512


Повторить результат из статьи не удалось. Улучшению может способствовать использование полной модели, а также большее время на обучение.